In [1]:
def read_wav(path):
    from scipy.io import wavfile
    samplerate, data = wavfile.read(path)
    return samplerate,data
def read_script(path,samplerate,video_name):
    from pandas import DataFrame
    tempstamps = []
    phrases = []
    with open(path,'r') as f:
        content = f.read().splitlines()
    for i in range(0,len(content),2):
        tempstamps.append(content[i])
        phrases.append(content[i+1])
    # convert timestamps to seconds
    timestamps = []
    for timestamp in tempstamps:
        vals = timestamp.split(':')
        if(len(vals) == 2):
            mm = int(vals[0])
            ss = int(vals[1])
            timestamps.append((mm*60)+ss)
        else:
            hh = int(vals[0])
            mm = int(vals[1])
            ss = int(vals[2])
            timestamps.append((hh*3600)+(mm*60)+ss)
    datastamps = [timestamp*samplerate for timestamp in timestamps]
    df = DataFrame([datastamps,datastamps[1:]+[1],phrases],index=['startstamps','endstamps','phrase']).T
    df = df[1:-1]
    df = df.reset_index(drop=True)
    df['phrase'].reset_index().rename({'index':'filename'},axis=1).to_csv(f'{video_name}/metadata.csv',index=False)
    return df
def split_audio(samplerate,data,content,video_name):
    from os import system
    from scipy.io import wavfile
    system(f'mkdir {video_name}/phrases')
    for i,row in enumerate(content.iloc):
        phrase = row['phrase']
        start = row['startstamps']
        end = row['endstamps']
        wavfile.write(f'{video_name}/phrases/{i}.wav',samplerate,data[start:end+15000])

In [2]:
video_name = f'numberphile'
samplerate,data = read_wav(f'{video_name}/audio.wav')
content = read_script(f'{video_name}/script.txt',samplerate,video_name)
split_audio(samplerate=samplerate,data=data,content=content,video_name=video_name)